In [19]:
# Nombre de matchs à compter pour le calcul du FDR
nb_fdr = 6
# Nombre de matchs à compter pour le calcul des stats avancées
nb_stats = 3

In [20]:
import requests
import pandas as pd
import numpy as np
import fonctions
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("Nombre de joueurs récupérés :", len(data["elements"]))
    # Exemple : afficher le nom du premier joueur
    print("Premier joueur :", data["elements"][0]["web_name"])
else:
    print("Erreur lors de la requête :", response.status_code)

Nombre de joueurs récupérés : 742
Premier joueur : Raya


In [21]:
fixtures_url = "https://fantasy.premierleague.com/api/fixtures/"
fixtures_response = requests.get(fixtures_url)
fixtures = fixtures_response.json()
print("Nombre de matchs à venir :", len(fixtures))

Nombre de matchs à venir : 380


In [22]:
df_teams = pd.DataFrame(data['teams'])[["id", "name", "short_name", "strength", "strength_overall_home", "strength_overall_away", "strength_attack_home", "strength_attack_away", "strength_defence_home", "strength_defence_away"]]

# Renommer pour clarté
df_teams = df_teams.rename(columns={
    'name': 'team_name'
})

# Ajout d'une colonne AvgStrenght
df_teams["avg_strenght"] = (df_teams["strength_overall_home"] + df_teams["strength_overall_away"]) / 2
df_teams.sort_values(by=["strength", "avg_strenght"] , inplace=True, ascending=False)

In [23]:
df_fixtures = pd.DataFrame(fixtures)

# Pour chaque match, récupérer les noms d'équipes à domicile et à l'extérieur
df_fixtures = df_fixtures.merge(df_teams, left_on='team_h', right_on='id', suffixes=('', '_home'))
df_fixtures = df_fixtures.merge(df_teams, left_on='team_a', right_on='id', suffixes=('', '_away'))

In [24]:
# Ajout du FDR pour chacune des équipes sur les 6 prochaines journées
# Supposons df_teams contient l'ID officiel de chaque club dans la colonne 'id'

team_fdr = []

for team_id in df_teams['id']:
    # Sélectionne les fixtures où l'équipe est à domicile
    home = df_fixtures[(df_fixtures['team_h'] == team_id) & (df_fixtures['finished'] == False)][['event', 'team_h_difficulty']]
    home = home.rename(columns={'team_h_difficulty': 'difficulty'})
    # Sélectionne les fixtures où l'équipe est à l'extérieur
    away = df_fixtures[(df_fixtures['team_a'] == team_id) & (df_fixtures['finished'] == False)][['event', 'team_a_difficulty']]
    away = away.rename(columns={'team_a_difficulty': 'difficulty'})
    # Concatène toutes les futures fixtures
    all_fixtures = pd.concat([home, away], ignore_index=True)
    # Trie par numéro de GW et garde les 6 prochaines
    next_six = all_fixtures.sort_values('event').head(nb_fdr)
    # Calcule la moyenne du FDR
    fdr_mean = next_six['difficulty'].mean()
    team_fdr.append(fdr_mean)

# Ajoute la colonne FDR moyen au DataFrame équipes
df_teams['fdr_next_6'] = team_fdr

In [25]:
df_teams

,id,team_name,short_name,strength,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,avg_strenght,fdr_next_6
11,12,Liverpool,LIV,5,1335,1355,1290,1330,1380,1380,1345.0,3.333333
0,1,Arsenal,ARS,4,1320,1325,1350,1350,1290,1300,1322.5,2.500000
12,13,Man City,MCI,4,1275,1315,1250,1250,1300,1380,1295.0,3.333333
6,7,Chelsea,CHE,4,1185,1245,1150,1190,1220,1300,1215.0,2.666667
14,15,Newcastle,NEW,4,1185,1245,1130,1170,1240,1320,1215.0,3.000000
1,2,Aston Villa,AVL,3,1125,1250,1110,1200,1140,1300,1187.5,3.166667
15,16,Nott'm Forest,NFO,3,1165,1205,1150,1230,1180,1180,1185.0,3.333333
3,4,Bournemouth,BOU,3,1150,1180,1100,1160,1200,1200,1165.0,3.166667
5,6,Brighton,BHA,3,1150,1175,1090,1140,1210,1210,1162.5,2.833333
4,5,Brentford,BRE,3,1120,1185,1080,1080,1160,1290,1152.5,3.166667


In [26]:
df_positions = pd.DataFrame(data['element_types'])[["id", "singular_name_short"]]
df_positions.set_index('id', inplace=True)
df_positions

,singular_name_short
id,
1,GKP
2,DEF
3,MID
4,FWD


In [27]:
# Selection des colonnes pertinentes 
df_players = pd.DataFrame(data['elements'])[['id', 'first_name', 'web_name', 'team', 'element_type', 'now_cost', 'total_points',
'ict_index', 'selected_by_percent', 'selected_rank', 'form', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event','minutes', 
'assists','goals_scored', 'expected_goals', 'expected_assists', 'expected_goal_involvements','expected_goals_per_90', 'saves_per_90',
'expected_assists_per_90', 'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90', 'goals_conceded_per_90',
'defensive_contribution_per_90', 'clean_sheets_per_90']]

In [28]:
# Conversion des types
df_players['ict_index'] = df_players['ict_index'].astype(float)
df_players['total_points'] = df_players['total_points'].astype(float)
df_players['minutes'] = df_players['minutes'].astype(float)
df_players['goals_scored'] = df_players['goals_scored'].astype(float)
df_players['form'] = df_players['form'].astype(float)
df_players['assists'] = df_players['assists'].astype(float)
df_players['expected_goals'] = df_players['expected_goals'].astype(float)
df_players['expected_assists'] = df_players['expected_assists'].astype(float)
df_players['expected_goal_involvements'] = df_players['expected_goal_involvements'].astype(float)
df_players['expected_goals_per_90'] = df_players['expected_goals_per_90'].astype(float)
df_players['saves_per_90'] = df_players['saves_per_90'].astype(float)
df_players['expected_assists_per_90'] = df_players['expected_assists_per_90'].astype(float)
df_players['expected_goal_involvements_per_90'] = df_players['expected_goal_involvements_per_90'].astype(float)
df_players['expected_goals_conceded_per_90'] = df_players['expected_goals_conceded_per_90'].astype(float)
df_players['goals_conceded_per_90'] = df_players['goals_conceded_per_90'].astype(float)
df_players['defensive_contribution_per_90'] = df_players['defensive_contribution_per_90'].astype(float) 
df_players['now_cost'] = df_players['now_cost'].astype(float)

# Joindre le nom de l'équipe sur la colonne 'team' (qui contient l'ID)
df_players = df_players.merge(df_teams[['id','team_name', 'short_name']], left_on='team', right_on='id', suffixes=('', '_team'))
# Joindre la position du joueur sur la colonne 'element_type' (ID)
df_players = df_players.merge(df_positions, left_on='element_type', right_on='id', suffixes=('', '_pos'))
# Ajout d'une colonne points total / prix
df_players.insert(loc=6, column="points_per_cost", value=df_players['total_points'] / df_players['now_cost'])
df_players.insert(loc=6, column="points_per_minutes", value=df_players['total_points'] / df_players['minutes'])
# Ajout d'une colonne GI
df_players.insert(loc=6, column="goal_involvements", value=df_players['assists'] + df_players['goals_scored'])
# Ajout d'une colonne GI_on_xGI
df_players.insert(loc=6, column="GI_on_xGI", value=df_players['goal_involvements'] / df_players['expected_goal_involvements'])
# Tri
df_players.sort_values(by=["points_per_cost", "total_points", "ict_index"] , inplace=True, ascending=False)
# Conversion
df_players['selected_by_percent'] = df_players['selected_by_percent'].astype(str).str.replace(',', '.').astype(float)

# Ajout de la colonne fdr_nex_6 depuis df_teams
df_players = df_players.merge(
    df_teams[['team_name', 'fdr_next_6']],
    on='team_name',
    how='left'
)

# Sélectionner les colonnes intéressantes pour plus de lisibilité
df_players = df_players[[
'id', 'first_name', 'web_name', 'short_name', 'team_name', 'singular_name_short', 'now_cost', 'total_points', 'points_per_cost', 'points_per_minutes', 'fdr_next_6',
'ict_index', 'selected_by_percent', 'selected_rank', 'form', 'minutes', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event',
'assists','goals_scored', 'goal_involvements', 'expected_goals', 'expected_assists', 'expected_goal_involvements', "GI_on_xGI", 'expected_goals_per_90', 'saves_per_90', 'expected_assists_per_90',
'expected_goal_involvements_per_90', 'expected_goals_conceded_per_90', 'goals_conceded_per_90',
'defensive_contribution_per_90', 'clean_sheets_per_90'    
]]

# Renommer pour clarté
df_players = df_players.rename(columns={
    'name': 'team_name',
    'short_name': 'team_short',
    'singular_name_short': 'position',
    'now_cost': 'price'
})

# Conversion des types
df_players['points_per_cost'] = df_players['points_per_cost'].astype(float)

# Gestion des NaN
for col in ['points_per_cost', 'ict_index', 'points_per_minutes', 'fdr_next_6']:
    df_players[col] = pd.to_numeric(df_players[col], errors='coerce').fillna(0)

# Enrichissement avec les stats avancées des n derniers matchs
df_players = fonctions.enrich_players_with_last_n_matches_data(df_players, n_last_matches=nb_stats)

# Ajout de la colonne total_points_last_per_xGI_last
df_players['total_points_last_per_xGI_last'] = df_players.apply(
    lambda row: row['total_points_last'] / row['xGI_last'] if row['xGI_last'] > 0 else 0,
    axis=1
)

# Remplacer les infinis par 0
df_players.replace([np.inf, -np.inf], 0, inplace=True) 

df_players.set_index("id", inplace=True)

In [29]:
df_players[df_players['position'] == 'MID'].head(20)

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,points_per_minutes,fdr_next_6,ict_index,selected_by_percent,selected_rank,form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,assists,goals_scored,goal_involvements,expected_goals,expected_assists,expected_goal_involvements,GI_on_xGI,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90,clean_sheets_per_90,minutes_last,xGI_last,xG_last,ict_last,total_points_last,total_points_last_per_xGI_last
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
200,Jaidon,Anthony,BUR,Burnley,MID,56.0,40.0,0.714286,0.075614,3.000000,48.2,4.0,91,6.2,529.0,503970,140777,91488,4545,1.0,4.0,5.0,2.16,0.61,2.77,1.805054,0.37,0.0,0.10,0.47,1.83,1.70,8.17,0.34,88.333333,0.263333,0.190000,7.966667,6.333333,24.050633
390,Ryan,Gravenberch,LIV,Liverpool,MID,56.0,35.0,0.625000,0.077778,3.333333,30.0,4.2,88,7.0,450.0,506220,80711,72634,6735,2.0,2.0,4.0,0.37,0.36,0.73,5.479452,0.07,0.0,0.07,0.14,1.03,1.00,9.60,0.40,90.000000,0.223333,0.110000,7.433333,8.666667,38.805970
660,Anton,Stach,LEE,Leeds,MID,50.0,31.0,0.620000,0.057407,2.666667,36.1,2.9,107,5.2,540.0,387834,22092,83772,5624,2.0,1.0,3.0,0.61,0.56,1.17,2.564103,0.10,0.0,0.09,0.19,1.22,1.50,8.33,0.33,90.000000,0.113333,0.053333,5.000000,6.000000,52.941176
205,Josh,Cullen,BUR,Burnley,MID,50.0,31.0,0.620000,0.062000,3.000000,29.0,2.0,135,3.2,500.0,300067,7545,128623,5274,1.0,1.0,2.0,0.08,0.57,0.65,3.076923,0.01,0.0,0.10,0.11,1.85,1.80,11.88,0.36,76.666667,0.016667,0.000000,2.800000,3.666667,220.000000
82,Antoine,Semenyo,BOU,Bournemouth,MID,78.0,48.0,0.615385,0.088889,3.166667,47.8,52.6,2,6.8,540.0,5538228,170368,487001,6939,2.0,4.0,6.0,2.99,0.31,3.30,1.818182,0.50,0.0,0.05,0.55,0.91,1.17,8.17,0.50,90.000000,0.450000,0.396667,7.066667,8.333333,18.518519
241,Moisés,Caicedo,CHE,Chelsea,MID,57.0,35.0,0.614035,0.064815,2.666667,28.4,12.0,36,5.5,540.0,1029043,28985,364769,33799,0.0,2.0,2.0,0.47,0.22,0.69,2.898551,0.08,0.0,0.04,0.12,1.33,1.33,14.17,0.33,90.000000,0.086667,0.043333,4.700000,6.000000,69.230769
237,Enzo,Enzo,CHE,Chelsea,MID,67.0,39.0,0.582090,0.073864,2.666667,43.8,13.3,29,6.5,528.0,1485838,75130,528930,19022,2.0,3.0,5.0,3.41,0.48,3.89,1.285347,0.58,0.0,0.08,0.66,1.35,1.36,5.80,0.34,90.000000,0.570000,0.530000,7.000000,4.666667,8.187135
668,Granit,Xhaka,SUN,Sunderland,MID,50.0,29.0,0.580000,0.053704,3.000000,30.5,3.1,104,5.5,540.0,183455,22967,159150,3748,3.0,0.0,3.0,0.12,0.63,0.75,4.000000,0.02,0.0,0.10,0.12,1.15,0.67,12.67,0.50,90.000000,0.166667,0.023333,5.400000,6.000000,36.000000
493,Sean,Longstaff,LEE,Leeds,MID,49.0,26.0,0.530612,0.067532,2.666667,28.1,0.3,321,6.0,385.0,23384,10989,13915,791,1.0,1.0,2.0,0.21,0.28,0.49,4.081633,0.05,0.0,0.07,0.12,1.09,1.17,10.75,0.23,87.333333,0.160000,0.070000,8.400000,6.333333,39.583333


In [30]:
df_players.to_parquet('df_players.parquet')
df_positions.to_parquet('df_positions.parquet')
df_teams.to_parquet('df_teams.parquet')

In [31]:
df_players.fillna(0, inplace=True)

In [32]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 742 entries, 683 to 229
Data columns (total 40 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   first_name                         742 non-null    object 
 1   web_name                           742 non-null    object 
 2   team_short                         742 non-null    object 
 3   team_name                          742 non-null    object 
 4   position                           742 non-null    object 
 5   price                              742 non-null    float64
 6   total_points                       742 non-null    float64
 7   points_per_cost                    742 non-null    float64
 8   points_per_minutes                 742 non-null    float64
 9   fdr_next_6                         742 non-null    float64
 10  ict_index                          742 non-null    float64
 11  selected_by_percent                742 non-null    float64
 1